<a href="https://colab.research.google.com/github/adithyarajagopal/Deep-Collective-Matri-Factorisation/blob/main/NUS_INTERN_NETWORK_EMBEDDING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False



    
if IN_COLAB:
    print("Downloading Colab files")
    ! shred -u setup_google_colab.py
    ! wget https://raw.githubusercontent.com/hse-aml/bayesian-methods-for-ml/master/setup_google_colab.py -O setup_google_colab.py
    import setup_google_colab
    setup_google_colab.load_data_week6()

--2020-10-07 13:38:08--  https://raw.githubusercontent.com/hse-aml/bayesian-methods-for-ml/master/setup_google_colab.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1232 (1.2K) [text/plain]
Saving to: ‘setup_google_colab.py’

setup_google_colab. 100%[===================>]   1.20K  --.-KB/s    in 0s      

2020-10-07 13:38:09 (65.9 MB/s) - ‘setup_google_colab.py’ saved [1232/1232]

https://raw.githubusercontent.com/hse-aml/bayesian-methods-for-ml/master/week6/w6_grader.py w6_grader.py


In [ ]:
! pip install GPy gpyopt xgboost

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


IMPORTING NECESSARY LIBRARIES


In [ ]:
!pip3 install ax-platform 

In [ ]:
import pandas as pd
import numpy as np
import os
import csv
from scipy.sparse import csr_matrix
cwd = os.getcwd()
from numpy import transpose
df2 = pd.read_csv('/content/node.dat', sep = '\t', header = None)
df1 = pd.read_csv('/content/link.dat',  sep = '\t', header = None)
print(df2.shape)

df2 = df2.head(10000)
print(df1.shape)
print(df2.shape)



(61149, 4)
(236458, 4)
(10000, 4)



PREPROCESSING FOR NODE EMBEDDINGS


HERE A SUBSET OF THE PUBMED GRAPH IS TAKEN, 10000 OUT OF 60000 NODEID'S AND CONSIDERED

ISOLATED NODES ARE DISCARDED

In [ ]:
df1.columns = ['nodeid1', 'nodeid2', 'linktype', 'linkweight']
df2.columns = ['nodeid', 'nodename', 'nodetype', 'nodeattribute']
df1 = df1[(df1['nodeid1']<10000) & (df1['nodeid2']<10000)]
print(df1.shape)
print(max(df2['nodeid']))
print(df1)
print(df2)
print(df2.shape)
df2.pop('nodename')





df = df1[['nodeid1','nodeid2']]



list_ = []


(7180, 4)
9999
        nodeid1  nodeid2  linktype  linkweight
17         8394     8772         5           1
18         8772     8394         5           1
40         8772     8772         5         106
59         3562     8321         4        2200
65         3562      963         4          48
...         ...      ...       ...         ...
236063     2579     3499         2           1
236254     4759     4956         2           1
236270     5033     5033         2           4
236421     2330     7476         2           1
236433     3256     7988         2           1

[7180 rows x 4 columns]
      nodeid  ...                                      nodeattribute
0          0  ...  0.02508,-0.036716,0.091492,-0.074665,0.022286,...
1          1  ...  -0.072251,-0.1242,0.027377,-0.049692,0.022852,...
2          2  ...  -0.85981,-1.632455,-0.238721,-0.527147,1.09448...
3          3  ...  -0.100137,-0.295236,0.204746,-0.043928,0.09525...
4          4  ...  0.005788,-0.080936,-0.038131,0.0

In [ ]:

node_list_1 = list(df1['nodeid1'])
node_list_2 = list(df1['nodeid2'])
node_list = node_list_1+ node_list_2
node_list = sorted(list(dict.fromkeys(node_list)))
print(node_list)

df2 = df2.set_index('nodeid')
print(df2)
df2 = df2.loc[node_list]
print(df2.shape)





df2.reset_index(inplace = True)
DF1 = df2[df2['nodetype'] == 0]
DF2 = df2[df2['nodetype'] == 1]
DF3 = df2[df2['nodetype'] == 2]
DF4 = df2[df2['nodetype'] == 3]

    
d_df1 = DF1['nodeattribute'].str.split(',', expand = True)
d_df2 = DF2['nodeattribute'].str.split(',', expand = True)
d_df3 = DF3['nodeattribute'].str.split(',', expand = True)
d_df4 = DF4['nodeattribute'].str.split(',', expand = True)
print(d_df1.shape)
print(d_df2.shape)
print(d_df3.shape)
print(d_df4.shape)

ID1 = DF1['nodeid']
ID2 = DF2['nodeid']
ID3 = DF3['nodeid']
ID4 = DF4['nodeid']
print('HELLO')


 




[1, 2, 10, 11, 18, 20, 22, 27, 28, 29, 30, 31, 35, 36, 38, 39, 42, 45, 46, 47, 48, 49, 55, 56, 57, 59, 62, 68, 70, 75, 83, 85, 88, 89, 90, 91, 94, 98, 99, 101, 102, 105, 109, 110, 111, 114, 119, 120, 121, 122, 125, 128, 130, 132, 133, 134, 138, 142, 143, 145, 147, 151, 154, 157, 161, 164, 167, 170, 171, 172, 174, 176, 180, 181, 182, 183, 184, 185, 189, 191, 193, 194, 195, 197, 198, 199, 202, 204, 209, 214, 220, 222, 223, 224, 225, 226, 227, 228, 229, 231, 238, 239, 240, 242, 244, 246, 248, 253, 256, 258, 260, 263, 264, 266, 268, 277, 278, 280, 282, 286, 287, 291, 293, 296, 297, 300, 301, 303, 305, 307, 308, 311, 313, 316, 317, 319, 330, 331, 333, 335, 337, 339, 343, 344, 347, 349, 350, 353, 360, 361, 364, 365, 366, 369, 371, 375, 380, 381, 384, 388, 390, 392, 393, 397, 398, 400, 403, 404, 405, 407, 410, 413, 415, 417, 418, 421, 423, 426, 427, 429, 430, 431, 433, 436, 439, 442, 444, 445, 447, 450, 452, 456, 457, 458, 461, 462, 463, 464, 465, 471, 472, 474, 484, 487, 489, 490, 491, 498, 

In [ ]:
list1 = [i for i in range(792)]
list_1 = [i for i in range(415)]
list2 = [i for i in range(1169)]
list3 = [i for i in range(1631)]
list4 = [i for i in range(178)]
list_3 = [i for i in range(3000)]
list_2 = [i for i in range(5000)]
dict1 = dict(zip(ID1, list1))
dict_1 = dict(zip(ID1, list_1))
dict2 = dict(zip(ID2, list2))
dict_2 = dict(zip(ID2, list_2))
dict3 = dict(zip(ID3, list3))
dict4 = dict(zip(ID4, list4))
dict_3 = dict(zip(ID3, list_3))

In [ ]:
'''print(len(link_1_1['nodeid2']))
print(A1.shape)
from numpy.linalg import inv
from numpy.linalg import matrix_power
link_1_list = list(link_1_1['nodeid1'])
link_2_list = list(link_1_1['nodeid2'])
link_list_1 = link_1_list + link_2_list
link_list_1 = sorted(list(dict.fromkeys(link_list_1)))
dict_1_ = dict(zip(link_list_1,range(0, len(link_list_1))))

dict1_1 = dict()
for index, row in link_1_1.iterrows():
  if (((row['nodeid1'], row['nodeid2'])) or ((row['nodeid2'], row['nodeid1']))) in dict1_1.keys():
    print('Hello')
    index = index + 1
    continue
  dict1_1[(dict_1_[row['nodeid1']]), (dict_1_[row['nodeid2']])] = row['linkweight']


print((dict1_1))
A1 = [0 for i in range(233) for j in range(233)]
A1 = np.matrix(A1).reshape(233,233)
for i in range(233):
  for j in range(233):
    if (i, j) in list(dict1_1.keys()):
      A1[i, j] = dict1_1[(i, j)]

print(A1)
D1 = [0 for i in range(233) for j in range(233)]
D1 = np.matrix(D1).reshape(233, 233)
D_1 = (np.sum(A1 , axis=1))
VOL_G1 = sum(D_1)
print(D_1)
for i in range(233):
  D1[i, i] = D_1[i]
print(D1)
P1 = np.dot(inv(D1), A1)
print(P1)
SUM11 = 0
for i in range(10):
  SUM11 = SUM11+ matrix_power(P1, i)
print(SUM11.shape)  
M11 = (np.dot(SUM11, inv(D1)))
print('Hellohello')
print(M11)
M11 = np.matrix(M11)
for i in range(M11.shape[0]):
  for j in range(M11.shape[1]):
    if M11[i, j] != 0:
       M11[i, j] = np.log((VOL_G1/100)*M11[i, j])


print(M11)

link_1_list2 = list(link_1_2['nodeid1'])
link_2_list2 = list(link_1_2['nodeid2'])

link_list_2 = link_1_list2 + link_2_list2
link_list_2 = sorted(list(dict.fromkeys(link_list_2)))

dict_2_ = dict(zip(link_list_2,range(0, len(link_list_2))))

dict1_2 = dict()
for index, row in link_1_2.iterrows():
  if (((row['nodeid1'], row['nodeid2'])) or ((row['nodeid2'], row['nodeid1']))) in dict1_2.keys():
    print('Hello')
    index = index + 1
    continue
  dict1_2[(dict_2_[row['nodeid1']]), (dict_2_[row['nodeid2']])] = row['linkweight']

A2 = [0 for i in range(631) for j in range(631)]
A2 = np.matrix(A2).reshape(631,631)
for i in range(631):
  for j in range(631):
    if (i, j) in list(dict1_2.keys()):
      A2[i, j] = dict1_2[(i, j)]

D2 = [0 for i in range(631) for j in range(631)]
D2 = np.matrix(D2).reshape(631, 631)
D_2 = (np.sum(A2 , axis=1))
VOL_G2 = sum(D_2)

for i in range(631):
  D2[i, i] = D_2[i]
print(D1)
P2 = np.dot(inv(D2), A2)

SUM12 = 0
for i in range(10):
  SUM12 = SUM12+ matrix_power(P2, i)
print(SUM12.shape)  
M12 = (np.dot(SUM12, inv(D2)))
print('Hellohello')
print(M11)
M12 = np.matrix(M12)
for i in range(M12.shape[0]):
  for j in range(M12.shape[1]):
    if M12[i, j] != 0:
       M12[i, j] = np.log((VOL_G2/100)*M12[i, j])

print(M12)

link_1_list3 = list(link_2_2['nodeid1'])
link_2_list3 = list(link_2_2['nodeid2'])
link_list_3 = link_1_list3 + link_2_list3
link_list_3 = sorted(list(dict.fromkeys(link_list_3)))
dict_3_ = dict(zip(link_list_3,range(0, len(link_list_3))))

dict2_2 = dict()
for index, row in link_2_2.iterrows():
  if (((row['nodeid1'], row['nodeid2'])) or ((row['nodeid2'], row['nodeid1']))) in dict2_2.keys():
    print('Hello')
    index = index + 1
    continue
  dict2_2[(dict_3_[row['nodeid1']]), (dict_3_[row['nodeid2']])] = row['linkweight']

A3 = [0 for i in range(469) for j in range(469)]
A3 = np.matrix(A3).reshape(469,469)
for i in range(469):
  for j in range(469):
    if (i, j) in list(dict2_2.keys()):
      A3[i, j] = dict2_2[(i, j)]

D3 = [0 for i in range(469) for j in range(469)]
D3 = np.matrix(D3).reshape(469, 469)
D_3 = (np.sum(A3 , axis=1))
VOL_G3 = sum(D_3)
print(D_3)
for i in range(469):
  D3[i, i] = D_3[i]
print(D1)
P3 = np.dot(inv(D3), A3)

SUM22 = 0
for i in range(10):
  SUM22 = SUM22+ matrix_power(P3, i)
print(SUM22.shape)  
M22 = (np.dot(SUM22, inv(D3)))
print('Hellohello')
print(M)
M22 = np.matrix(M22)
for i in range(M22.shape[0]):
  for j in range(M22.shape[1]):
    if M22[i, j] != 0:
       M22[i, j] = np.log((VOL_G3/100)*M22[i, j])


print(M22)

link_1_list4 = list(link_3_1['nodeid1'])
link_2_list4 = list(link_3_1['nodeid2'])
link_list_4 = link_1_list4 + link_2_list4
link_list_4 = sorted(list(dict.fromkeys(link_list_4)))
dict_4_ = dict(zip(link_list_4,range(0, len(link_list_4))))

dict3_1 = dict()
for index, row in link_3_1.iterrows():
  if (((row['nodeid1'], row['nodeid2'])) or ((row['nodeid2'], row['nodeid1']))) in dict3_1.keys():
    print('Hello')
    index = index + 1
    continue
  dict3_1[(dict_4_[row['nodeid1']]), (dict_4_[row['nodeid2']])] = row['linkweight']

A4 = [0 for i in range(956) for j in range(956)]
A4 = np.matrix(A4).reshape(956,956)
for i in range(956):
  for j in range(956):
    if (i, j) in list(dict3_1.keys()):
      A4[i, j] = dict3_1[(i, j)]

D4 = [0 for i in range(956) for j in range(956)]
D4 = np.matrix(D4).reshape(956, 956)
D_4 = (np.sum(A4 , axis=1))
VOL_G4 = sum(D_4)
print(dict_4_)
for i in range(956):
  D4[i, i] = D_4[i]
print(D4)
P4 = np.dot(inv(D4), A4)

SUM31 = 0
for i in range(10):
  SUM31 = SUM31+ matrix_power(P4, i)
print(SUM31.shape)  
M31 = (np.dot(SUM31, inv(D4)))
print('Hellohello')
print(M)
M31 = np.matrix(M31)
for i in range(M31.shape[0]):
  for j in range(M31.shape[1]):
    if M31[i, j] != 0:
       M31[i, j] = np.log((VOL_G4/100)*M31[i, j])


link_1_list5 = list(link_3_2['nodeid1'])
link_2_list5 = list(link_3_2['nodeid2'])
link_list_5 = link_1_list5 + link_2_list5
link_list_5 = sorted(list(dict.fromkeys(link_list_5)))
dict_5_ = dict(zip(link_list_5,range(0, len(link_list_5))))

dict3_2 = dict()
for index, row in link_3_2.iterrows():
  if (((row['nodeid1'], row['nodeid2'])) or ((row['nodeid2'], row['nodeid1']))) in dict3_2.keys():
    print('Hello')
    index = index + 1
    continue
  dict3_2[(dict_5_[row['nodeid1']]), (dict_5_[row['nodeid2']])] = row['linkweight']

A5 = [0 for i in range(1392) for j in range(1392)]
A5 = np.matrix(A5).reshape(1392,1392)
for i in range(1392):
  for j in range(1392):
    if (i, j) in list(dict3_2.keys()):
      A5[i, j] = dict3_2[(i, j)]

D5 = [0 for i in range(1392) for j in range(1392)]
D5 = np.matrix(D5).reshape(1392, 1392)
D_5 = (np.sum(A5 , axis=1))
VOL_G5 = sum(D_5)
print(dict_5_)
for i in range(1392):
  D5[i, i] = D_5[i]
print(D5)
P5 = np.dot(inv(D5), A5)

SUM32 = 0
for i in range(10):
  SUM32 = SUM32+ matrix_power(P5, i)
print(SUM32.shape)  
M32 = (np.dot(SUM32, inv(D5)))
print('Hellohello')
print(M)
M32 = np.matrix(M32)
for i in range(M32.shape[0]):
  for j in range(M32.shape[1]):
    if M32[i, j] != 0:
       M32[i, j] = np.log((VOL_G5/100)*M32[i, j])

link_1_list6 = list(link_3_3['nodeid1'])
link_2_list6 = list(link_3_3['nodeid2'])
link_list_6 = link_1_list6 + link_2_list6
link_list_6 = sorted(list(dict.fromkeys(link_list_6)))
dict_6_ = dict(zip(link_list_6,range(0, len(link_list_6))))

dict3_3 = dict()
for index, row in link_3_3.iterrows():
  if (((row['nodeid1'], row['nodeid2'])) or ((row['nodeid2'], row['nodeid1']))) in dict3_3.keys():
    print('Hello')
    index = index + 1
    continue
  dict3_3[(dict_6_[row['nodeid1']]), (dict_6_[row['nodeid2']])] = row['linkweight']

A6 = [0 for i in range(1141) for j in range(1141)]
A6 = np.matrix(A6).reshape(1141,1141)
for i in range(1141):
  for j in range(1141):
    if (i, j) in list(dict3_3.keys()):
      A6[i, j] = dict3_3[(i, j)]

D6 = [0 for i in range(1141) for j in range(1141)]
D6 = np.matrix(D6).reshape(1141, 1141)
D_6 = (np.sum(A6 , axis=1))
VOL_G6 = sum(D_6)
print(dict_6_)
for i in range(1141):
  D6[i, i] = D_6[i]
print(D6)
P6 = np.dot(inv(D6), A6)

SUM33 = 0
for i in range(10):
  SUM33 = SUM33+ matrix_power(P6, i)
print(SUM33.shape)  
M33 = (np.dot(SUM33, inv(D6)))
print('Hellohello')
print(M)
M33 = np.matrix(M33)
for i in range(M33.shape[0]):
  for j in range(M33.shape[1]):
    if M33[i, j] != 0:
       M33[i, j] = np.log((VOL_G6/100)*M33[i, j])

link_1_list7 = list(link_3_4['nodeid1'])
link_2_list7 = list(link_3_4['nodeid2'])
link_list_7 = link_1_list7 + link_2_list7
link_list_7 = sorted(list(dict.fromkeys(link_list_7)))
dict_7_ = dict(zip(link_list_7,range(0, len(link_list_7))))

dict3_4 = dict()
for index, row in link_3_4.iterrows():
  if (((row['nodeid1'], row['nodeid2'])) or ((row['nodeid2'], row['nodeid1']))) in dict3_4.keys():
    print('Hello')
    index = index + 1
    continue
  dict3_4[(dict_7_[row['nodeid1']]), (dict_7_[row['nodeid2']])] = row['linkweight']

A7 = [0 for i in range(209) for j in range(209)]
A7 = np.matrix(A7).reshape(209,209)
for i in range(209):
  for j in range(209):
    if (i, j) in list(dict3_4.keys()):
      A7[i, j] = dict3_4[(i, j)]

D7 = [0 for i in range(209) for j in range(209)]
D7 = np.matrix(D7).reshape(209, 209)
D_7 = (np.sum(A7 , axis=1))
VOL_G7 = sum(D_7)
print(dict_7_)
for i in range(209):
  D7[i, i] = D_7[i]
print(D7)
P7 = np.dot(inv(D7), A7)

SUM34 = 0
for i in range(10):
  SUM34 = SUM34+ matrix_power(P7, i)
print(SUM34.shape)  
M34 = (np.dot(SUM34, inv(D7)))
print('Hellohello')
print(M)
M34 = np.matrix(M34)
for i in range(M34.shape[0]):
  for j in range(M34.shape[1]):
    if M34[i, j] != 0:
       M34[i, j] = np.log((VOL_G7/100)*M34[i, j])

link_1_list8 = list(link_4_1['nodeid1'])
link_2_list8 = list(link_4_1['nodeid2'])
link_list_8 = link_1_list8 + link_2_list8
link_list_8 = sorted(list(dict.fromkeys(link_list_8)))
dict_8_ = dict(zip(link_list_8,range(0, len(link_list_8))))

dict4_1 = dict()
for index, row in link_4_1.iterrows():
  if (((row['nodeid1'], row['nodeid2'])) or ((row['nodeid2'], row['nodeid1']))) in dict4_1.keys():
    print('Hello')
    index = index + 1
    continue
  dict4_1[(dict_8_[row['nodeid1']]), (dict_8_[row['nodeid2']])] = row['linkweight']

A4 = [0 for i in range(118) for j in range(118)]
A4 = np.matrix(A4).reshape(118,118)
for i in range(118):
  for j in range(118):
    if (i, j) in list(dict4_1.keys()):
      A8[i, j] = dict4_1[(i, j)]

D8 = [0 for i in range(118) for j in range(118)]
D8 = np.matrix(D8).reshape(118, 118)
D_8 = (np.sum(A8 , axis=1))
VOL_G8 = sum(D_8)
print(dict_8_)
for i in range(118):
  D8[i, i] = D_8[i]
print(D8)
P8 = np.dot(inv(D8), A8)

SUM41 = 0
for i in range(10):
  SUM41 = SUM41+ matrix_power(P8, i)
print(SUM41.shape)  
M41 = (np.dot(SUM41, inv(D8)))
print('Hellohello')
print(M)
M41 = np.matrix(M41)
for i in range(M41.shape[0]):
  for j in range(M41.shape[1]):
    if M41[i, j] != 0:
       M41[i, j] = np.log((VOL_G8/100)*M41[i, j])


link_1_list9 = list(link_4_2['nodeid1'])
link_2_list9 = list(link_4_2['nodeid2'])
link_list_9 = link_1_list9 + link_2_list9
link_list_9 = sorted(list(dict.fromkeys(link_list_9)))
dict_9_ = dict(zip(link_list_9,range(0, len(link_list_9))))

dict4_2 = dict()
for index, row in link_4_2.iterrows():
  if (((row['nodeid1'], row['nodeid2'])) or ((row['nodeid2'], row['nodeid1']))) in dict4_2.keys():
    print('Hello')
    index = index + 1
    continue
  dict4_2[(dict_9_[row['nodeid1']]), (dict_9_[row['nodeid2']])] = row['linkweight']

A9 = [0 for i in range(118) for j in range(118)]
A9 = np.matrix(A9).reshape(118,118)
for i in range(118):
  for j in range(118):
    if (i, j) in list(dict4_2.keys()):
      A9[i, j] = dict4_2[(i, j)]

D9 = [0 for i in range(118) for j in range(118)]
D9 = np.matrix(D9).reshape(118, 118)
D_9 = (np.sum(A9 , axis=1))
VOL_G9 = sum(D_9)
print(dict_9_)
for i in range(118):
  D_9[i, i] = D_9[i]
print(D9)
P9 = np.dot(inv(D9), A9)

SUM42 = 0
for i in range(10):
  SUM42 = SUM42+ matrix_power(P9, i)
print(SUM31.shape)  
M42 = (np.dot(SUM42, inv(D9)))
print('Hellohello')
print(M)
M42 = np.matrix(M42)
for i in range(M42.shape[0]):
  for j in range(M42.shape[1]):
    if M42[i, j] != 0:
       M42[i, j] = np.log((VOL_G9/100)*M42[i, j])


link_1_list10 = list(link_4_4['nodeid1'])
link_2_list10 = list(link_4_4['nodeid2'])
link_list_10 = link_1_list10 + link_2_list10
link_list_10 = sorted(list(dict.fromkeys(link_list_10)))
dict_10_ = dict(zip(link_list_10,range(0, len(link_list_10))))

dict4_4 = dict()
for index, row in link_4_4.iterrows():
  if (((row['nodeid1'], row['nodeid2'])) or ((row['nodeid2'], row['nodeid1']))) in dict4_4.keys():
    print('Hello')
    index = index + 1
    continue
  dict4_4[(dict_10_[row['nodeid1']]), (dict_10_[row['nodeid2']])] = row['linkweight']

A10 = [0 for i in range(15) for j in range(15)]
A10 = np.matrix(A10).reshape(15,15)
for i in range(15):
  for j in range(15):
    if (i, j) in list(dict4_4.keys()):
      A10[i, j] = dict4_4[(i, j)]

D10 = [0 for i in range(15) for j in range(15)]
D10 = np.matrix(D10).reshape(15, 15)
D_10 = (np.sum(A10 , axis=1))
VOL_G10 = sum(D_10)
print(dict_10_)
for i in range(15):
  D10[i, i] = D_10[i]
print(D10)
P10 = np.dot(inv(D10), A10)

SUM44 = 0
for i in range(10):
  SUM44 = SUM44+ matrix_power(P10, i)
print(SUM31.shape)  
M44 = (np.dot(SUM44, inv(D10)))
print('Hellohello')
print(M)
M44 = np.matrix(M44)
for i in range(M44.shape[0]):
  for j in range(M44.shape[1]):
    if M44[i, j] != 0:
       M44[i, j] = np.log((VOL_G10/100)*M44[i, j])
'''



'''
i_j = []
for i in range((A1.shape[0])):
  for j in range((A1.shape[1])):
    if A1[i, j] != 0:
      if (i, j) not in dict1_1.keys():
        i_j.append((i, j))

        j = j+1
        continue
      A1[i, j] = dict1_1[i, j]
print(A1)
print(i_j)'''

In [ ]:
from numpy.linalg import inv
from numpy.linalg import matrix_power
link_1_list = list(df1['nodeid1'])
link_2_list = list(df1['nodeid2'])
link_list_1 = link_1_list + link_2_list
link_list_1 = sorted(list(dict.fromkeys(link_list_1)))
dict_1_ = dict(zip(link_list_1,range(0, len(link_list_1))))

dict1_1 = dict()
for index, row in df1.iterrows():
  if (((row['nodeid1'], row['nodeid2'])) or ((row['nodeid2'], row['nodeid1']))) in dict1_1.keys():
    print('Hello')
    index = index + 1
    continue
  dict1_1[(dict_1_[row['nodeid1']]), (dict_1_[row['nodeid2']])] = row['linkweight']


print((dict1_1))
A1 = [0 for i in range(3770) for j in range(3770)]
A1 = np.matrix(A1).reshape(3770,3770)
for (i, j) in list(dict1_1.keys()):

  A1[i, j] = dict1_1[(i, j)]

print(A1)
D1 = [0 for i in range(3770) for j in range(3770)]
D1 = np.matrix(D1).reshape(3770, 3770)
D_1 = (np.sum(A1 , axis=1))
VOL_G1 = sum(D_1)
print(D_1)
for i in range(3770):
  if D_1[i] != 0:
    D1[i,i] = (1/D_1[i])
  
print(D1)

A1 = np.matrix(A1)
A1_ = [0 for i in range(3700) for j in range(3700)]
A1_ = np.matrix(A1_)
A1_ = A1_.reshape(3700, 3700)
A1_ = A1*0.95

eye1 = np.eye(3770)
Sgra = (eye1 - np.dot(A1_, (D1)))
Sgra = inv(Sgra)
Sgra = np.dot(Sgra, A1_)
print(Sgra)
link_weight_list = []
for index, row in df1.iterrows():
  link_weight_list.append(Sgra[dict_1_[row['nodeid1']], dict_1_[row['nodeid2']]])




df1['linkweight1'] = list(link_weight_list)


Hello
Hello
Hello
Hello
Hello
{(3189, 3326): 1, (3326, 3189): 1, (3326, 3326): 106, (1378, 3171): 2200, (1378, 369): 48, (1378, 333): 310, (1378, 2076): 3, (2988, 2742): 8, (2513, 278): 9, (983, 2326): 1, (2326, 983): 1, (851, 786): 2, (2630, 2941): 3, (2630, 162): 29, (3326, 3732): 79, (2436, 276): 112, (276, 2436): 112, (1378, 3732): 64, (333, 3171): 11, (312, 2530): 1, (2530, 312): 1, (1378, 983): 626, (983, 1378): 626, (1378, 1771): 7, (1771, 1378): 7, (3701, 1683): 2, (1735, 249): 4, (2406, 1654): 9, (1931, 727): 2, (1464, 3326): 1, (3326, 1464): 1, (2282, 2282): 2, (418, 2929): 2, (2042, 854): 11, (3051, 3326): 61, (3326, 3051): 61, (1726, 3215): 4, (983, 661): 3, (661, 983): 3, (2728, 3105): 2, (738, 218): 1, (218, 738): 1, (1378, 1359): 2, (1108, 2482): 38, (1378, 2433): 1, (2433, 1378): 1, (1378, 2759): 423, (3371, 1332): 8, (1332, 3371): 8, (2376, 2887): 6, (3562, 346): 1, (346, 3562): 1, (1680, 98): 1, (98, 1680): 1, (3160, 2042): 2, (2042, 3160): 2, (1378, 1953): 287, (1953

In [ ]:
df1 = df1.drop('linkweight', axis = 1)
g_df1 = df1.groupby('linktype')
for key, item in g_df1:
    list_.append(g_df1.get_group(key))


link_1_1 = list_[0].sort_values('nodeid1')
link_1_2 = list_[1].sort_values('nodeid1')
link_2_2 = list_[2].sort_values('nodeid1')
link_3_1 = list_[3].sort_values('nodeid1')
link_3_2 = list_[4].sort_values('nodeid1')
link_3_3 = list_[5].sort_values('nodeid1')
link_3_4 = list_[6].sort_values('nodeid1')
link_4_1 = list_[7].sort_values('nodeid1')
link_4_2 = list_[8].sort_values('nodeid1')
link_4_4 = list_[9].sort_values('nodeid1')    

In [ ]:
for item in list_:
    item.sort_values('nodeid1')
    

matrix_1_1 = csr_matrix(np.zeros([792, 792], dtype = 'int8'))
matrix_1_2 = csr_matrix(np.zeros([792,1169], dtype = 'int8'))
matrix_2_2 = csr_matrix(np.zeros([1169,1169], dtype = 'int8'))
matrix_3_1 = csr_matrix(np.zeros([1631,792 ], dtype = 'int8'))
matrix_3_2 = csr_matrix(np.zeros([1631,1169], dtype = 'int8'))

In [ ]:
matrix_3_3 = csr_matrix(np.zeros([1631,1631], dtype = 'int8'))
matrix_3_4 = csr_matrix(np.zeros([1631,178], dtype = 'int8'))
matrix_4_1 = csr_matrix(np.zeros([178,792], dtype = 'int8'))
matrix_4_2 = csr_matrix(np.zeros([178,1169], dtype = 'int8'))
matrix_4_4 = csr_matrix(np.zeros([178,178], dtype = 'int8'))



list1 = [i for i in range(792)]
list_1 = [i for i in range(415)]
list2 = [i for i in range(1169)]
list3 = [i for i in range(1631)]
list4 = [i for i in range(178)]
list_3 = [i for i in range(3000)]
list_2 = [i for i in range(5000)]
dict1 = dict(zip(ID1, list1))
dict_1 = dict(zip(ID1, list_1))
dict2 = dict(zip(ID2, list2))
dict_2 = dict(zip(ID2, list_2))
dict3 = dict(zip(ID3, list3))
dict4 = dict(zip(ID4, list4))
dict_3 = dict(zip(ID3, list_3))

In [ ]:
matrix_1_1 = matrix_1_1.todense()
matrix_1_2 = matrix_1_2.todense()
matrix_2_2 = matrix_2_2.todense()
matrix_3_1 = matrix_3_1.todense()
matrix_3_2 = matrix_3_2.todense()
matrix_3_3 = matrix_3_3.todense()
matrix_3_4 = matrix_3_4.todense()
matrix_4_1 = matrix_4_1.todense()
matrix_4_2 = matrix_4_2.todense()
matrix_4_4 = matrix_4_4.todense()

In [ ]:
import networkx as nx
col_1_3 = list(link_3_1.columns)
col_1_4 = list(link_4_1.columns)
col_1_3[0], col_1_3[1] = col_1_3[1], col_1_3[0]
col_1_4[0], col_1_4[1] = col_1_4[1], col_1_4[0]
print('Hello World')
link_3_1_new = link_3_1[col_1_3]
link_4_1_new = link_4_1[col_1_4]
print(link_3_1_new)
print(link_4_1_new)
M1 = pd.concat([link_1_1, link_1_2, link_3_1_new, link_4_1_new])
print('m1')
col_2_1 = list(link_1_2.columns)
col_2_1[0], col_2_1[1] = col_2_1[1], col_2_1[0]
link_1_2_new = link_1_2[col_2_1]
col_2_3 = list(link_3_2.columns)
col_2_3[0], col_2_3[1] = col_2_3[1], col_2_3[0]
link_3_2_new = link_3_2[col_2_3]
col_2_4 = list(link_4_2.columns)
col_2_4[0], col_2_4[1] = col_2_4[1], col_2_4[0]
link_4_2_new = link_4_2[col_2_4]
M2 = pd.concat([link_1_2_new, link_2_2, link_3_2_new, link_4_2_new])
print('m2')
M3 = pd.concat([link_3_1, link_3_2, link_3_3, link_3_4])
print('m3')
col_4_3 = list(link_3_4.columns)
col_4_3[0], col_4_3[1] = col_4_3[1], col_4_3[0]
link_3_4_new = link_3_4[col_4_3]
M4 = pd.concat([link_4_1, link_4_2, link_3_4_new, link_4_4])
print('m4')

G1 = nx.from_pandas_edgelist(M1, "nodeid1", "nodeid2", edge_attr = 'linkweight', create_using=  nx.Graph())
G2 = nx.from_pandas_edgelist(M2, 'nodeid1', 'nodeid2', edge_attr = 'linkweight', create_using = nx.Graph())
G3 = nx.from_pandas_edgelist(M3, 'nodeid1', 'nodeid2', edge_attr = 'linkweight', create_using = nx.Graph())
G4 = nx.from_pandas_edgelist(M4, 'nodeid1', 'nodeid2', edge_attr = 'linkweight', create_using = nx.Graph())
A1 = nx.to_numpy_matrix(G1)
print(A1)
A2 = nx.to_numpy_matrix(G2)
print(A2)
A3 = nx.to_numpy_matrix(G3)
print(A3)
A4 = nx.to_numpy_matrix(G4)
print(A4)
import numpy as np
from numpy.linalg import inv
from numpy.linalg import matrix_power
K1 = []
K2 = []
K3 = []
K4 = []
for (node, val) in G1.degree():
  K1.append((val))
for (node, val) in G2.degree():
  K2.append(val)  
for (node, val) in G3.degree():
  K3.append(val)
for (node, val) in G4.degree():
  K4.append(val) 
VOL_G1 = 2*(sum(K1))
VOL_G2 = 2*(sum(K2))
VOL_G3 = 2*(sum(K3))
VOL_G4 = 2*(sum(K4))
K1 = np.array(K1)
K2 = np.array(K2)
K3 = np.array(K3)     
K4 = np.array(K4)

D1 = np.diag(K1)
print(D1.shape)
D2 = np.diag(K2)
print(D2.shape)
D3 = np.diag(K3)
print(D3.shape)
D4 = np.diag(K4)
print(D4.shape)
P1 = np.dot(inv(D1), A1)
print('P1')
P2 = np.dot(inv(D2), A2)
print('P2')
P3 = np.dot(inv(D3), A3)
print('P3')
P4 = np.dot(inv(D4), A4)
print('P4')
b_hyper_parameter = 10
T_hyper_parameter = 10
SUM11 = 0
K11 = []
SUM22 = 0
K22 = []
SUM33 = 0
K33 = []
SUM44 = 0
K44 = []
for i in range(1,10):
  SUM11 = SUM11+ matrix_power(P1, i)
  K11.append(matrix_power(P1, i))
for i in range(1, 10):
  SUM22 = SUM22 + matrix_power(P2, i)
  K22.append(matrix_power(P2, i))
for i in range(1, 10):
  SUM33 = SUM33 + matrix_power(P3, i)
  K33.append(matrix_power(P3, i))
for i in range(1, 10):
  SUM44 = SUM44 + matrix_power(P4, i)
  K44.append(matrix_power(P4, i))  

Pr1 = (VOL_G1/100)*(np.dot(SUM11, inv(D1)))
Pr2 = (VOL_G2/100)*(np.dot(SUM22, inv(D2)))
Pr3 = (VOL_G3/100)*(np.dot(SUM33, inv(D3)))
Pr4 = (VOL_G4/100)*(np.dot(SUM44, inv(D4)))

PR1 = np.log(Pr1)
PR2 = np.log(Pr2)
PR3 = np.log(Pr3)
PR4 = np.log(Pr4)


In [ ]:
print(G1.nodes())
Pr1.shape

CONSTRUCTION OF MATRICES

In [ ]:
for index, row in link_1_1.iterrows():
   
    if row['nodeid1'] not in dict1.keys():
        
        index = index + 1
        continue
       
        
        link_1_1.drop(index, axis = 0)
    if row['nodeid2'] not in dict1.keys():
        
        index = index + 1
        continue
       
        link_1_1.drop(index, axis = 0)
      
    
    matrix_1_1[dict1[row['nodeid1']], dict1[row['nodeid2']]] = row['linkweight1']
   
print(matrix_1_1)
        
for index, row in link_1_2.iterrows():
    if row['nodeid1'] not in dict1.keys():
        
        index = index + 1
        continue
        link_1_2.drop(index, axis = 0)
        
    if row['nodeid2'] not in dict2.keys():
        index = index + 1
        continue
        link_1_2.drop(index, axis = 0)
        
    matrix_1_2[dict1[row['nodeid1']], dict2[row['nodeid2']]] = row['linkweight1']


print(matrix_1_2)


for index, row in link_2_2.iterrows():
    if row['nodeid1'] not in dict2.keys():
        index = index +1
        continue
        link_2_2.drop( index, axis = 0)
        
    if row['nodeid2'] not in dict2.keys():
        index = index + 1
        continue
        link_2_2.drop(index, axis = 0)
        
    matrix_2_2[dict2[row['nodeid1']], dict2[row['nodeid2']]] = row['linkweight1']
    
print(matrix_2_2)

[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 3 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
for index, row in link_3_1.iterrows():
    if row['nodeid1'] not in dict3.keys():
        index = index +1
        continue
        link_3_1.drop( index, axis = 0)
        
    if row['nodeid2'] not in dict1.keys():
        index = index + 1
        continue
        link_3_1.drop(index, axis = 0)
        
    matrix_3_1[dict3[row['nodeid1']], dict1[row['nodeid2']]] = row['linkweight1']
    
print(matrix_3_1)


for index, row in link_3_2.iterrows():
    if row['nodeid1'] not in dict3.keys():
        index = index +1
        continue
        link_3_2.drop( index, axis = 0)
        
    if row['nodeid2'] not in dict2.keys():
        index = index + 1
        continue
        link_3_2.drop(index, axis = 0)
        
    matrix_3_2[dict3[row['nodeid1']], dict2[row['nodeid2']]] = row['linkweight1']
    
print(matrix_3_2)

for index, row in link_3_3.iterrows():
    if row['nodeid1'] not in dict3.keys():
        index = index +1
        continue
        link_3_3.drop( index, axis = 0)
        
    if row['nodeid2'] not in dict3.keys():
        index = index + 1
        continue
        link_3_3.drop(index, axis = 0)
        
    matrix_3_3[dict3[row['nodeid1']], dict3[row['nodeid2']]] = row['linkweight1']
    
print(matrix_3_3)


for index, row in link_3_4.iterrows():
    if row['nodeid1'] not in dict3.keys():
        index = index +1
        continue
        link_3_4.drop( index, axis = 0)
        
    if row['nodeid2'] not in dict4.keys():
        index = index + 1
        continue
        link_3_4.drop(index, axis = 0)
        
    matrix_3_4[dict3[row['nodeid1']], dict4[row['nodeid2']]] = row['linkweight1']
    
print(matrix_3_4)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
for index, row in link_4_1.iterrows():
    if row['nodeid1'] not in dict4.keys():
        index = index +1
        continue
        link_4_1.drop( index, axis = 0)
        
    if row['nodeid2'] not in dict1.keys():
        index = index + 1
        continue
        link_4_1.drop(index, axis = 0)
        
    matrix_4_1[dict4[row['nodeid1']], dict1[row['nodeid2']]] = row['linkweight1']
    
print(matrix_4_1)

for index, row in link_4_2.iterrows():
    if row['nodeid1'] not in dict4.keys():
        index = index +1
        continue
        link_4_2.drop( index, axis = 0)
        
    if row['nodeid2'] not in dict2.keys():
        index = index + 1
        continue
        link_4_2.drop(index, axis = 0)
        
    matrix_4_2[dict4[row['nodeid1']], dict2[row['nodeid2']]] = row['linkweight1']
    
print(matrix_4_2)

for index, row in link_4_4.iterrows():
    if row['nodeid1'] not in dict4.keys():
        index = index +1
        continue
        link_4_4.drop( index, axis = 0)
        
    if row['nodeid2'] not in dict4.keys():
        index = index + 1
        continue
        link_4_4.drop(index, axis = 0)
        
    matrix_4_4[dict4[row['nodeid1']], dict4[row['nodeid2']]] = row['linkweight1']
    
print(matrix_4_4)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
d_df1 = np.asmatrix(d_df1)
d_df2 = np.asmatrix(d_df2)
d_df3 = np.asmatrix(d_df3)
d_df4 = np.asmatrix(d_df4)
matrix_1_1 = np.asmatrix(matrix_1_1, dtype = 'int8')
matrix_1_2 = np.asmatrix(matrix_1_2, dtype = 'int8')
matrix_2_2 = np.asmatrix(matrix_2_2, dtype = 'int8')
matrix_3_1 = np.asmatrix(matrix_3_1, dtype = 'int8')
matrix_3_2 = np.asmatrix(matrix_3_2, dtype = 'int8')
matrix_3_3 = np.asmatrix(matrix_3_3, dtype = 'int8')
matrix_3_4 = np.asmatrix(matrix_3_4, dtype = 'int8')
matrix_4_1 = np.asmatrix(matrix_4_1, dtype = 'int8')
matrix_4_2 = np.asmatrix(matrix_4_2, dtype = 'int8')
matrix_4_4 = np.asmatrix(matrix_4_4, dtype = 'int8')
print(d_df1.shape)
print(d_df2)
print(d_df3)
print(d_df4)

(792, 200)
[['-0.016865' '0.004677' '0.066797' ... '0.053799' '0.007856' '0.010209']
 ['0.263025' '-0.112156' '0.206528' ... '-0.150739' '0.006579'
  '-0.163309']
 ['0.023108' '-0.110109' '0.139279' ... '-0.013597' '0.01412' '0.049978']
 ...
 ['-0.608391' '-0.042003' '-0.672979' ... '0.140407' '-0.624376'
  '0.340738']
 ['0.740398' '-0.389508' '0.113731' ... '-0.443919' '0.080166'
  '-0.232449']
 ['0.085177' '0.048592' '0.053117' ... '0.00756' '-0.098562' '-0.001695']]
[['-0.072251' '-0.1242' '0.027377' ... '0.030818' '-0.08409' '0.11262']
 ['0.436725' '0.518297' '0.093618' ... '-0.125465' '0.119006' '0.163113']
 ['-0.092507' '-0.074675' '0.062198' ... '-0.066435' '-0.045652'
  '0.119261']
 ...
 ['-0.635149' '-0.034425' '0.2176' ... '-0.180225' '-0.05104' '-0.03692']
 ['-0.007735' '-0.031073' '-0.026947' ... '0.034765' '0.008106'
  '-0.005443']
 ['-0.205861' '-0.031513' '0.115919' ... '0.333031' '0.135602'
  '-0.122662']]
[['0.030015' '-0.184471' '0.088601' ... '0.153053' '-0.308266' '

In [ ]:
print(matrix_1_1.shape)
print(matrix_1_2.shape)
print(matrix_2_2.shape)
print(matrix_3_1.shape)
print(matrix_3_2.shape)
print(matrix_3_3.shape)
print(matrix_3_4.shape)
print(matrix_4_1.shape)
print(matrix_4_2.shape)
print(matrix_4_4.shape)


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
matrix_1_1 = scaler.fit_transform(matrix_1_1)
matrix_1_2 = scaler.fit_transform(matrix_1_2)
matrix_2_2 = scaler.fit_transform(matrix_2_2)
matrix_3_1 = scaler.fit_transform(matrix_3_1)
matrix_3_2 = scaler.fit_transform(matrix_3_2)
matrix_3_3 = scaler.fit_transform(matrix_3_3)
matrix_3_4 = scaler.fit_transform(matrix_3_4)
matrix_4_1 = scaler.fit_transform(matrix_4_1)
matrix_4_2 = scaler.fit_transform(matrix_4_2)
matrix_4_4 = scaler.fit_transform(matrix_4_4)
d_df1 = scaler.fit_transform(d_df1)
d_df2 = scaler.fit_transform(d_df2)
d_df3 = scaler.fit_transform(d_df3)
d_df4 = scaler.fit_transform(d_df4)
'''
for i in range(matrix_1_1.shape[0]):
  for j in range((matrix_1_1.shape[1])):
    matrix_1_1[i, j] = float(matrix_1_1[i, j] + 1)
print(matrix_1_1)

for i in range((matrix_1_2.shape[0])):
  for j in range((matrix_1_2.shape[1])):
    matrix_1_2[i, j] = float(matrix_1_2[i, j] + 1)
print(matrix_1_2)

for i in range((matrix_2_2.shape[0])):
  for j in range((matrix_2_2.shape[1])):
    matrix_2_2[i, j] = float(matrix_2_2[i, j] + 1)
print(matrix_2_2)

for i in range((matrix_3_2.shape[0])):
  for j in range((matrix_3_2.shape[1])):
    matrix_3_2[i, j] = float(matrix_3_2[i, j] + 1)
print(matrix_3_2)

for i in range((matrix_3_3.shape[0])):
  for j in range((matrix_3_3.shape[1])):
    matrix_3_3[i, j] = float(matrix_3_3[i, j] + 1)
print(matrix_3_3)

for i in range((matrix_3_1.shape[0])):
  for j in range((matrix_3_1.shape[1])):
    matrix_3_1[i, j] = float(matrix_3_1[i, j] + 1)
print(matrix_3_1)

for i in range((matrix_3_4.shape[0])):
  for j in range((matrix_3_4.shape[1])):
    matrix_3_4[i, j] = float(matrix_3_4[i, j] + 1)
print(matrix_3_4)

for i in range((matrix_4_1.shape[0])):
  for j in range((matrix_4_1.shape[1])):
    matrix_4_1[i, j] = float(matrix_4_1[i, j] + 1)
print(matrix_4_1)

for i in range((matrix_4_2.shape[0])):
  for j in range((matrix_4_2.shape[1])):
    matrix_4_2[i, j] = float(matrix_4_2[i, j] + 1)
print(matrix_4_2)

for i in range((matrix_4_4.shape[0])):
  for j in range((matrix_4_4.shape[1])):
    matrix_4_4[i, j] = float(matrix_4_4[i, j] + 1)
print(matrix_4_4)

for i in range((d_df1.shape[0])):
  for j in range((d_df1.shape[1])):
    d_df1[i, j] = float(float(d_df1[i, j]) + 1)
print(d_df1)

for i in range((d_df2.shape[0])):
  for j in range((d_df2.shape[1])):
    d_df2[i, j] = float(float(d_df2[i, j]) + 1)
print(d_df2)

for i in range((d_df3.shape[0])):
  for j in range((d_df3.shape[1])):
    d_df3[i, j] = float(float(d_df3[i, j]) + 1)
print(d_df3)

for i in range((d_df4.shape[0])):
  for j in range((d_df4.shape[1])):
    d_df4[i, j] = float(float(d_df4[i, j]) + 1)
print(d_df4)


try:
  matrix_1_1 = np.log(matrix_1_1)
  matrix_1_2 = np.log(matrix_1_2)
  matrix_2_2 = np.log(matrix_2_2)
  matrix_3_1 = np.log(matrix_3_1)
  matrix_3_2 = np.log(matrix_3_2)
  matrix_3_3 = np.log(matrix_3_3)
  matrix_3_4 = np.log(matrix_3_4)
  matrix_4_1 = np.log(matrix_4_1)
  matrix_4_2 = np.log(matrix_4_2)
  matrix_4_4 = np.log(matrix_4_4)
  d_df1 = np.log(d_df1)
  d_df2 = np.log(d_df2)
  d_df3 = np.log(d_df3)
  d_df4 = np.log(d_df4)
  print((matrix_1_1, matrix_1_2, matrix_2_2, matrix_3_1, matrix_3_2, matrix_3_3, matrix_3_4, matrix_4_1, matrix_4_2, matrix_4_4, d_df1, d_df2, d_df3, d_df4))
except Exception as e:
  print('Ignore')


'''



(792, 792)
(792, 1169)
(1169, 1169)
(1631, 792)
(1631, 1169)
(1631, 1631)
(1631, 178)
(178, 792)
(178, 1169)
(178, 178)


"\nfor i in range(matrix_1_1.shape[0]):\n  for j in range((matrix_1_1.shape[1])):\n    matrix_1_1[i, j] = float(matrix_1_1[i, j] + 1)\nprint(matrix_1_1)\n\nfor i in range((matrix_1_2.shape[0])):\n  for j in range((matrix_1_2.shape[1])):\n    matrix_1_2[i, j] = float(matrix_1_2[i, j] + 1)\nprint(matrix_1_2)\n\nfor i in range((matrix_2_2.shape[0])):\n  for j in range((matrix_2_2.shape[1])):\n    matrix_2_2[i, j] = float(matrix_2_2[i, j] + 1)\nprint(matrix_2_2)\n\nfor i in range((matrix_3_2.shape[0])):\n  for j in range((matrix_3_2.shape[1])):\n    matrix_3_2[i, j] = float(matrix_3_2[i, j] + 1)\nprint(matrix_3_2)\n\nfor i in range((matrix_3_3.shape[0])):\n  for j in range((matrix_3_3.shape[1])):\n    matrix_3_3[i, j] = float(matrix_3_3[i, j] + 1)\nprint(matrix_3_3)\n\nfor i in range((matrix_3_1.shape[0])):\n  for j in range((matrix_3_1.shape[1])):\n    matrix_3_1[i, j] = float(matrix_3_1[i, j] + 1)\nprint(matrix_3_1)\n\nfor i in range((matrix_3_4.shape[0])):\n  for j in range((matrix_3_4.

In [ ]:
print(matrix_1_1)
print(matrix_1_2)
print(matrix_2_2)
print(matrix_3_1)
print(matrix_3_2)
print(matrix_3_3)
print(matrix_3_4)
print(matrix_4_1)
print(matrix_4_2)
print(matrix_4_4)
print(d_df1)
print(d_df2)
print(d_df3)
print(d_df4)

[[28.12472222  0.          0.         ...  0.          0.
  -0.03555591]
 [-0.03555591  0.          0.         ...  0.          0.
  -0.03555591]
 [-0.03555591  0.          0.         ...  0.          0.
  -0.03555591]
 ...
 [-0.03555591  0.          0.         ...  0.          0.
  -0.03555591]
 [-0.03555591  0.          0.         ...  0.          0.
  -0.03555591]
 [-0.03555591  0.          0.         ...  0.          0.
  28.12472222]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[-2.92602868e-02  0.00000000e+00  0.00000000e+00 ... -2.92602868e-02
   0.00000000e+00  0.00000000e+00]
 [-2.92602868e-02  0.00000000e+00  0.00000000e+00 ... -2.92602868e-02
   0.00000000e+00  0.00000000e+00]
 [-2.92602868e-02  0.00000000e+00  0.00000000e+00 ... -2.92602868e-02
   0.00000000e+00  0.00000000e+00]
 ...
 [-2.92602868e-02  0.00000000e+00  0.00000000e+00 ...  3.41760150e+01
   0.00000

D_DF1, D_DF2, D_DF3, D_DF4 ARE FEATURE MATRICES

MATRIX_A_B DESCRIBES THE RELATIONSHIP BETWEEN PAIR CONSISTING OF NODE TYPE A AND NODE TYPE B

In [ ]:
import sys
sys.path.append("..")
import pprint

import pickle as pkl
import time
import itertools
import os
from dcmf import dcmf

from dcmf_bo import dcmf_bo
import ax
from ax import RangeParameter, ChoiceParameter, FixedParameter
from ax import ParameterType, SearchSpace
from ax.service.managed_loop import optimize

num_folds = 1

In [ ]:
#d_df3 = d_df3[:-1 , :]
#print(d_df3.shape)

In [ ]:
import torch
torch.cuda.is_available()

True

OBTAINING DCMF EMBEDDINGS

In [ ]:
G = {
    "e1":["X1","X2","X4", "X8", "X11"],\
    "e2":["X2","X3","X5", "X9","X12"],\
    "e3":["X4", "X5", "X6", "X7", "X13"],\
    "e4":["X7", "X8", "X9", "X10", "X14"],\
     "e5": ["X11"],\
     "e6": ["X12"],\
     "e7" : ["X13"],\
     "e8" : ["X14"]
     
    }



X_data = {
    "X1":matrix_1_1,\
    "X2":matrix_1_2,\
    "X3":matrix_2_2,\
    "X4":matrix_3_1,\
    "X5":matrix_3_2, "X6": matrix_3_3, "X7": matrix_3_4, "X8": matrix_4_1, "X9": matrix_4_2, "X10": matrix_4_4, "X11" : d_df1, "X12" : d_df2, "X13" : d_df3, "X14" : d_df4 }

X_meta = {
    "X1":["e1","e1"],\
    "X2":["e1","e2"],\
    "X3":["e2","e2"],\
    "X4":["e3","e1"],\
    "X5":["e3","e2"], "X6": ["e3", "e3"], "X7" : ["e3", "e4"], "X8":["e4", "e1"], "X9": ["e4", "e2"], "X10": ["e4", "e4"], "X11" : ["e1", "e5"], "X12" : ["e2", "e6"], "X13" : ["e3", "e7"], "X14" : ["e4", "e8"]}

HERE K IS THE EMBEDDING SIZE, WHICH IS 50 

In [ ]:
kf = 0.5
k = 50
e_actf = "tanh"
d_actf = "tanh"
is_linear_last_enc_layer = False
is_linear_last_dec_layer = False
num_chunks = 2

In [ ]:
learning_rate = 0.001
weight_decay = 0.05
max_epochs = 500
convg_thres = 0.1

In [ ]:
is_pretrain=True
pretrain_thres= 0.1
max_pretrain_epochs = 2

In [ ]:
val_metric = "rmse"
is_val_transpose = False
at_k = 10

In [ ]:
is_gpu = True
gpu_ids = "1"

In [ ]:
best_criterion = "loss" 
num_bo_steps = 500000
initial_design_size = 5

In [ ]:
def run_dcmf(parameterization):
    #hyper-parameters that are selected using ax
    learning_rate = parameterization["learning_rate"]
    weight_decay = parameterization["weight_decay"]
    #
    dcmf_model = dcmf(G, X_data, X_meta,\
                num_chunks=num_chunks,k=k, kf=kf, e_actf=e_actf, d_actf=d_actf,\
                learning_rate=learning_rate, weight_decay=weight_decay, convg_thres=convg_thres, max_epochs=max_epochs,\
                is_gpu=is_gpu,gpu_ids=gpu_ids,is_pretrain=is_pretrain, pretrain_thres=pretrain_thres,\
                max_pretrain_epochs=max_pretrain_epochs,val_metric=val_metric,\
                is_val_transpose=is_val_transpose, at_k=at_k,\
                is_linear_last_enc_layer=is_linear_last_enc_layer,is_linear_last_dec_layer=is_linear_last_dec_layer,num_val_sets=num_folds)
    #
    dcmf_model.fit()
    print("#")
    print("dcmf_model.out_dict_info: ")
    pprint.pprint(dcmf_model.out_dict_info)
    print("#")
    #
    out_dict = {}
    out_dict["loss"] = (dcmf_model.out_dict_info["loss_all_folds_avg_sum"], 0.0)
    return out_dict

In [ ]:
best_parameters, values, experiment, model = optimize(
    parameters=[
        {
            "name": "weight_decay",
            "type": "range",
            "bounds": [1e-6, 1e-2],
            "value_type": "float",  # Optional, defaults to inference from type of "bounds".
            "log_scale": False,  # Optional, defaults to False.
        },
        {
            "name": "learning_rate",
            "type": "range",
            "bounds": [1e-7, 1e-5], #mortality1y
            #"bounds": [1e-5, 1e-4], #diag
            "value_type": "float",  # Optional, defaults to inference from type of "bounds".
            "log_scale": False,  # Optional, defaults to False.
        }
        # {
        #     "name": "convg_thres",
        #     "type": "range",
        #     "bounds": [1e-5, 1e-3], #diag
        #     "value_type": "float",  # Optional, defaults to inference from type of "bounds".
        #     "log_scale": False,  # Optional, defaults to False.
        # },
        # {
        #     "name": "num_layers",
        #     "type": "choice",
        #     #"values": [0, 1, 2],
        #     "values": [2,2],
        #     "value_type": "int"
        # },
        # {
        #     "name": "k",
        #     "type": "choice",
        #     #"values": [50, 100, 150, 200],
        #     #"values": [50,100,200],
        #     "value_type": "int"
        # }
        # {
        #     "name": "actf",
        #     "type": "choice",
        #     "values": ["tanh", "sigma"],
        #     "value_type": "str"
        # }
        # {
        #     "name": "num_layers",
        #     "type": "choice",
        #     "values": [1,2],
        #     "value_type": "int"
        # }
    ],
    experiment_name="dcmf_bo",
    objective_name="loss",
    evaluation_function=run_dcmf,
    minimize=True,  # Optional, defaults to False.
    #parameter_constraints=["k%2 <= 0"],  # Optional.
    #outcome_constraints=["loss >= 0"],  # Optional.
    total_trials=2, # Optional.
)

[INFO 10-07 13:42:14] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 5 arms, GPEI for subsequent arms], generated 0 arm(s) so far). Iterations after 5 will take longer to generate due to model-fitting.
[INFO 10-07 13:42:14] ax.service.managed_loop: Started full optimization with 2 steps.
[INFO 10-07 13:42:14] ax.service.managed_loop: Running optimization trial 1...


dcmf_base.__init__ - start
dcmf_base.__init__ - end
val_metric:  rmse
at_k:  10
is_val_transpose:  False
#
dCMF:
---
#
dCMF: 
#
learning_rate:  7.702615290880204e-06
weight_decay:  0.0014462092408388855
convg_thres:  0.1
max_epochs:  500
isPretrain:  True
pretrain_thres:  0.1
max_pretrain_epochs:  2
num_chunks:  2
k:  50
kf:  0.5
e_actf:  tanh
d_actf:  tanh
is_gpu:  True
gpu_ids:  1
num entities:  8
num matrices:  14
num_val_sets:  1
X_val #matrices:  0
val_metric (used only if X_val #matrices > 0):  rmse
at_k (used only if X_val #matrices > 0 and val_metric is r@k or p@k):  10
is_val_transpose:  False
is_linear_last_enc_layer:  False
is_linear_last_dec_layer:  False
#
## fold_num:  1  ##
dcmf_base.__init__ - start
dcmf_base.__init__ - end
val_metric:  rmse
at_k:  10
is_val_transpose:  False
#
dCMF: 
#
learning_rate:  7.702615290880204e-06
weight_decay:  0.0014462092408388855
convg_thres:  0.1
max_epochs:  500
isPretrain:  True
pretrain_thres:  0.1
max_pretrain_epochs:  2
num_chunks:  

[INFO 10-07 13:42:19] ax.service.managed_loop: Running optimization trial 2...


epoch:  2  total loss L:  27.319260597229004  Took  0.4  secs.
**train converged**
#
dcmf.fit - end
#
dcmf_model.out_dict_info: 
{'E': 8,
 'M': 14,
 'loss_all_folds': {'1': [0.8840658962726593,
                          0.9202503859996796,
                          1.0704075694084167,
                          0.7124446332454681,
                          2.004258155822754,
                          2.0035394430160522,
                          2.00239098072052,
                          2.016843557357788,
                          0.6740932464599609,
                          0.5176565200090408,
                          0.8012714982032776,
                          1.359588861465454,
                          1.3093912601470947,
                          0.9114593863487244,
                          1.2166489958763123,
                          0.13571834564208984,
                          0.12861960381269455,
                          0.5513730049133301,
                          2

In [ ]:
print("experiment.trials: ")
pprint.pprint(experiment.trials)
print("#")

print("best_parameters: ")
print(best_parameters)
print("#")

print("values[0]: ")
print(values[0])
print("#")

for idx in experiment.trials.keys():
    trial =  experiment.trials[idx]
    print("obj: ",round(trial.objective_mean,4)," params: ",trial.arm.parameters)
print("#")


learning_rate = best_parameters["learning_rate"]
weight_decay = best_parameters["weight_decay"]

experiment.trials: 
{0: Trial(experiment_name='dcmf_bo', index=0, status=TrialStatus.COMPLETED, arm=Arm(name='0_0', parameters={'weight_decay': 0.0014462092408388855, 'learning_rate': 7.702615290880204e-06})),
 1: Trial(experiment_name='dcmf_bo', index=1, status=TrialStatus.COMPLETED, arm=Arm(name='1_0', parameters={'weight_decay': 0.000592032627977431, 'learning_rate': 9.733881253004076e-06}))}
#
best_parameters: 
{'weight_decay': 0.0014462092408388855, 'learning_rate': 7.702615290880204e-06}
#
values[0]: 
{'loss': 27.319262005388737}
#
obj:  27.3193  params:  {'weight_decay': 0.0014462092408388855, 'learning_rate': 7.702615290880204e-06}
obj:  27.5823  params:  {'weight_decay': 0.000592032627977431, 'learning_rate': 9.733881253004076e-06}
#


In [ ]:
dcmf_model = dcmf(G, X_data, X_meta,\
            num_chunks=num_chunks,k=k, kf=kf, e_actf=e_actf, d_actf=d_actf,\
            learning_rate=learning_rate, weight_decay=weight_decay, convg_thres=convg_thres, max_epochs=max_epochs,\
            is_gpu=is_gpu,gpu_ids=gpu_ids,is_pretrain=is_pretrain, pretrain_thres=pretrain_thres,\
            max_pretrain_epochs=max_pretrain_epochs,val_metric=val_metric,\
            is_val_transpose=is_val_transpose, at_k=at_k,\
            is_linear_last_enc_layer=is_linear_last_enc_layer,is_linear_last_dec_layer=is_linear_last_dec_layer,num_val_sets=num_folds)

dcmf_base.__init__ - start
dcmf_base.__init__ - end
val_metric:  rmse
at_k:  10
is_val_transpose:  False
#
dCMF:
---
#
dCMF: 
#
learning_rate:  7.702615290880204e-06
weight_decay:  0.0014462092408388855
convg_thres:  0.1
max_epochs:  500
isPretrain:  True
pretrain_thres:  0.1
max_pretrain_epochs:  2
num_chunks:  2
k:  50
kf:  0.5
e_actf:  tanh
d_actf:  tanh
is_gpu:  True
gpu_ids:  1
num entities:  8
num matrices:  14
num_val_sets:  1
X_val #matrices:  0
val_metric (used only if X_val #matrices > 0):  rmse
at_k (used only if X_val #matrices > 0 and val_metric is r@k or p@k):  10
is_val_transpose:  False
is_linear_last_enc_layer:  False
is_linear_last_dec_layer:  False
#


In [ ]:
'''dcmf_bo_model = dcmf_bo(G, X_data, X_meta,\
            num_chunks=num_chunks,k=k, kf=kf, e_actf=e_actf, d_actf=d_actf,\
            learning_rate=None, weight_decay=None, convg_thres=None, max_epochs=max_epochs,\
            is_gpu=is_gpu,gpu_ids=gpu_ids,is_pretrain=is_pretrain, pretrain_thres=pretrain_thres,\
            max_pretrain_epochs=max_pretrain_epochs,val_metric=val_metric,\
            is_val_transpose=is_val_transpose, at_k=at_k, best_criterion=best_criterion,\
            is_linear_last_enc_layer=is_linear_last_enc_layer,is_linear_last_dec_layer=is_linear_last_dec_layer,num_val_sets=num_folds)
'''
            

'dcmf_bo_model = dcmf_bo(G, X_data, X_meta,            num_chunks=num_chunks,k=k, kf=kf, e_actf=e_actf, d_actf=d_actf,            learning_rate=None, weight_decay=None, convg_thres=None, max_epochs=max_epochs,            is_gpu=is_gpu,gpu_ids=gpu_ids,is_pretrain=is_pretrain, pretrain_thres=pretrain_thres,            max_pretrain_epochs=max_pretrain_epochs,val_metric=val_metric,            is_val_transpose=is_val_transpose, at_k=at_k, best_criterion=best_criterion,            is_linear_last_enc_layer=is_linear_last_enc_layer,is_linear_last_dec_layer=is_linear_last_dec_layer,num_val_sets=num_folds)\n'

In [ ]:
dcmf_model.fit()

## fold_num:  1  ##
dcmf_base.__init__ - start
dcmf_base.__init__ - end
val_metric:  rmse
at_k:  10
is_val_transpose:  False
#
dCMF: 
#
learning_rate:  7.702615290880204e-06
weight_decay:  0.0014462092408388855
convg_thres:  0.1
max_epochs:  500
isPretrain:  True
pretrain_thres:  0.1
max_pretrain_epochs:  2
num_chunks:  2
k:  50
kf:  0.5
e_actf:  tanh
d_actf:  tanh
is_gpu:  True
gpu_ids:  1
num entities:  8
num matrices:  14
num_val_sets:  1
X_val #matrices:  0
val_metric (used only if X_val #matrices > 0):  rmse
at_k (used only if X_val #matrices > 0 and val_metric is r@k or p@k):  10
is_val_transpose:  False
is_linear_last_enc_layer:  False
is_linear_last_dec_layer:  False
#
dcmf - model construction - start
__input_transformation - start
#
concatenated-matrix construction...
e_id:  e1
X_id_list:  ['X1', 'X2', 'X4', 'X8', 'X11']
X_id:  X1
X[X_id].shape:  (792, 792)
X_id:  X2
X[X_id].shape:  (792, 1169)
X_id:  X4
X[X_id].shape:  (1631, 792)
X_id:  X8
X[X_id].shape:  (178, 792)
X_id:  

In [ ]:
#dcmf_model.fit()

In [ ]:
dcmf_model.out_dict_U['1'].keys()


dict_keys(['e1', 'e2', 'e3', 'e4', 'e5', 'e6', 'e7', 'e8'])

In [ ]:
dcmf_model.out_dict_X_prime['1'].keys()


dict_keys(['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14'])

In [ ]:
dcmf_model.out_dict_U['1']['e2'].shape

torch.Size([1169, 50])

PERFORMING LINK PREDICTION

In [ ]:
import pandas as pd
import numpy as np
import random
import networkx as nx
from tqdm import tqdm
import re
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

CONSTRUCTION OF ADJACENCY MATRIX FROM THE GRAPH

In [ ]:
df = df1[['nodeid1', 'nodeid2', 'linkweight1']]
print(df)

df['link'] = 1

print(df)
G = nx.from_pandas_edgelist(df, "nodeid1", "nodeid2", create_using=nx.Graph())
print(G)
print(G.number_of_edges())
node_list_1 = list(df['nodeid1'])
node_list_2 = list(df['nodeid2'])
node_list = node_list_1+ node_list_2
node_list = list(dict.fromkeys(node_list))
adj_G = nx.to_numpy_matrix(G, nodelist = node_list)
print(adj_G.shape)

        nodeid1  nodeid2  linkweight1
17         8394     8772      0.95000
18         8772     8394      0.95000
40         8772     8772    162.97250
59         3562     8321   2182.95750
65         3562      963     45.60000
...         ...      ...          ...
236063     2579     3499      9.74359
236254     4759     4956      9.74359
236270     5033     5033      3.80000
236421     2330     7476      0.95000
236433     3256     7988      0.95000

[7180 rows x 3 columns]
        nodeid1  nodeid2  linkweight1  link
17         8394     8772      0.95000     1
18         8772     8394      0.95000     1
40         8772     8772    162.97250     1
59         3562     8321   2182.95750     1
65         3562      963     45.60000     1
...         ...      ...          ...   ...
236063     2579     3499      9.74359     1
236254     4759     4956      9.74359     1
236270     5033     5033      3.80000     1
236421     2330     7476      0.95000     1
236433     3256     7988      0.950

CREATING A SET OF UNCONNECTED NODE PAIRS FROM THE ADJACENCY MATRIX

In [ ]:
all_unconnected_pairs = []

# traverse adjacency matrix
offset = 0
for i in tqdm(range(adj_G.shape[0])):
  for j in range(offset,adj_G.shape[1]):
    if i != j:
      
    
        if adj_G[i,j] == 0:
          all_unconnected_pairs.append([node_list[i],node_list[j]])

  offset = offset + 1

100%|██████████| 3770/3770 [00:15<00:00, 237.21it/s] 


In [ ]:
print(len(all_unconnected_pairs))
node_1_unlinked = [i[0] for i in all_unconnected_pairs]
node_2_unlinked = [i[1] for i in all_unconnected_pairs]

data = pd.DataFrame({'nodeid1':node_1_unlinked, 
                     'nodeid2':node_2_unlinked})

# add target variable 'link'
data['link'] = 0
print(len(node_list))

7099483
3770


In [ ]:
ListOfNodes = G.nodes()
NumberofNodes = G.number_of_nodes()
NumberofEdges = G.number_of_edges()
print(NumberofEdges)

5392


CREATING A BALANCED DATAFRAME CONSISTING OF EQUAL PROPORTION OF CONNECTED AND UNCONNECTED NODE PAIRS

In [ ]:
data1 = data.sample(7180)
print(data1)
data2 = df[['nodeid1', 'nodeid2', 'link']]
print(data2)

DATA = data1.append(data2, ignore_index = True)
print(DATA)


         nodeid1  nodeid2  link
5991829     7207     5925     0
1412359     9804     9134     0
6357582     5625     1733     0
436340      5022     1239     0
1528221     6377      640     0
...          ...      ...   ...
4051927     7009     1117     0
1502072     6089     1673     0
5817396     1735     1922     0
913994      7440     8287     0
925495      3796     7480     0

[7180 rows x 3 columns]
        nodeid1  nodeid2  link
17         8394     8772     1
18         8772     8394     1
40         8772     8772     1
59         3562     8321     1
65         3562      963     1
...         ...      ...   ...
236063     2579     3499     1
236254     4759     4956     1
236270     5033     5033     1
236421     2330     7476     1
236433     3256     7988     1

[7180 rows x 3 columns]
       nodeid1  nodeid2  link
0         7207     5925     0
1         9804     9134     0
2         5625     1733     0
3         5022     1239     0
4         6377      640     0
...        ...

HERE E1,E2,E3 AND E4 ARE THE MATRICES CONTAINING DCMF EMBEDDINGS FOR THE 4 NODE TYPES

In [ ]:


E1 = dcmf_model.out_dict_U['1']['e1'].detach().cpu().numpy()
E2 = dcmf_model.out_dict_U['1']['e2'].detach().cpu().numpy()
E3 = dcmf_model.out_dict_U['1']['e3'].detach().cpu().numpy()
E4 = dcmf_model.out_dict_U['1']['e4'].detach().cpu().numpy()
print(E1.shape)
print(E2.shape)
print(E3.shape)
print(E4.shape)

(792, 50)
(1169, 50)
(1631, 50)
(178, 50)


In [ ]:
print(ID1.index.values)
print(ID1)


print(ID2.index.values)
print(ID3.index.values)
print(ID4.index.values)

[   1    3    6   14   21   25   26   28   37   38   44   47   52   64
   69   71   73   74   76   81   83   84   87   90   91   92   99  103
  104  107  110  116  120  134  137  141  146  147  162  175  184  186
  193  194  199  213  214  218  221  230  233  236  239  242  243  244
  251  252  263  270  272  277  280  285  288  291  293  294  297  301
  302  309  310  313  317  321  323  327  333  334  336  345  347  351
  353  355  358  359  362  364  368  378  391  397  411  415  424  426
  428  449  452  453  456  463  464  466  468  476  477  479  482  483
  484  514  517  518  519  520  521  523  527  531  532  535  537  542
  552  559  560  563  571  578  582  585  589  593  595  599  603  610
  615  621  634  635  645  649  650  653  664  665  667  689  692  693
  696  701  702  707  708  711  720  723  732  734  735  736  738  740
  747  749  752  758  760  792  806  807  812  813  830  831  833  835
  836  842  845  847  851  858  869  872  876  878  882  885  898  907
  922 

In [ ]:
ONES = np.ones(792)
ZEROS = np.zeros(1169)
ZEROS2 = np.zeros(1631)
for i in range(len(ZEROS2)):
  ZEROS2[i] = 2
#print(ZEROS2)
ZEROS3 = np.zeros(178)
for i in range(len(ZEROS3)):
  ZEROS3[i] = 3

TYPE1 = ZEROS
TYPE2 = ONES
TYPE3 = ZEROS2
TYPE4 = ZEROS3
#print(TYPE1.shape)
#print(TYPE2.shape)
#print(TYPE3.shape)
#print(TYPE4.shape)
DICT1 = dict(zip(ID1, TYPE1))
DICT2 = dict(zip( ID2, TYPE2))
DICT3 = dict(zip(ID3, TYPE3))
DICT4 = dict(zip(ID4, TYPE4))
DICT_ = {**DICT1, **DICT2, **DICT3, **DICT4}
dict_ = {**dict1, **dict2, **dict3, **dict4}

#print(len(dict_))
#print(len(DICT_))
list1_embold = []
list2_embold = []
listno_embold = []

for index, row in DATA.iterrows():
  

  if (row['nodeid1'] in dict1.keys()):
    list1_embold.append(E1[dict1[row['nodeid1']], :])
  elif (row['nodeid1'] in dict2.keys()):
    list1_embold.append(E2[dict2[row['nodeid1']], :]) 
  elif (row['nodeid1'] in dict3.keys()):
    list1_embold.append(E3[dict3[row['nodeid1']], :])
  elif (row['nodeid1'] in dict4.keys()):
    list1_embold.append(E4[dict4[row['nodeid1']], :])     
  else:
    listno_embold.append(row['nodeid1'])  
  if  (row['nodeid2'] in dict1.keys()):
    list2_embold.append(E1[dict1[row['nodeid2']], :]) 
 
  elif (row['nodeid2'] in dict2.keys()):
    list2_embold.append(E2[dict2[row['nodeid2']], :])
  
  elif (row['nodeid2'] in dict3.keys()):
    list2_embold.append(E3[dict3[row['nodeid2']], :])
  
  elif (row['nodeid2'] in dict4.keys()):
    list2_embold.append(E4[dict4[row['nodeid2']], :]) 
  else:
    listno_embold.append(row['nodeid2'])             

print(np.array(list1_embold).shape)
print(np.array(list2_embold).shape)
print(np.array(listno_embold).shape)


(14360, 50)
(14360, 50)
(0,)


CONSTRUCTION OF FEATURE VECTORS BY COMPUTING HADAMARD PRODUCT OF THE TWO LIST CONTAINING NODE EMBEDDINGS

FITTING THE MODEL USING SVM, LINEAR KERNEL

In [ ]:
#list1_embold = list1_embold[: len(list2_embold)] 

EMB_DATA = pd.DataFrame({'embedid1': list1_embold, 'embedid2': list2_embold})


EMB_DATA['Had_prod'] = EMB_DATA['embedid1']*EMB_DATA['embedid2']
#DATA = DATA.iloc[ : -2]
print(DATA.shape)

print(len(EMB_DATA['Had_prod']))
print(len(DATA['link']))
EMB_DATA1 = pd.DataFrame({'Had_EMB_LIST' : list(EMB_DATA['Had_prod']), 'Link' : list(DATA['link'])})
print(len(list(EMB_DATA['Had_prod'])))
print(len(list(DATA['link'])))





dict_final = EMB_DATA1
dict_final_1 = dict_final.sample(8480)
dict_final_2 = dict_final.sample(8480)
dict_final_3 = dict_final.sample(8480)
dict_final_4 = dict_final.sample(8480)
dict_final_5 = dict_final.sample(8480)
X1 = dict_final_1.Had_EMB_LIST.apply(pd.Series).to_numpy()
X2 = dict_final_2.Had_EMB_LIST.apply(pd.Series).to_numpy()
X3 = dict_final_3.Had_EMB_LIST.apply(pd.Series).to_numpy()
X4 = dict_final_4.Had_EMB_LIST.apply(pd.Series).to_numpy()
X5 = dict_final_5.Had_EMB_LIST.apply(pd.Series).to_numpy()
Y1 = list(dict_final_1['Link'])
Y2 = list(dict_final_2['Link'])
Y3 = list(dict_final_3['Link'])
Y4 = list(dict_final_4['Link'])
Y5 = list(dict_final_5['Link'])
from sklearn.svm import LinearSVC
clf1 = LinearSVC(max_iter = 3000)
clf2 = LinearSVC(max_iter = 3000)
clf3 = LinearSVC(max_iter = 3000)
clf4 = LinearSVC(max_iter = 3000)
clf5 = LinearSVC(max_iter = 3000)
clf1.fit(X1, Y1)
clf2.fit(X2, Y2)
clf3.fit(X3, Y3)
clf4.fit(X4, Y4)
clf5.fit(X5, Y5)
print(Y1)











(14360, 3)
14360
14360
14360
14360
[0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1

SIMILIAR PREPROCESSING FOR TEST SET

In [ ]:
df_test = pd.read_csv('/content/link.dat.test', sep = '\t', header = None)

df_test.columns = ['nodeid1', 'nodeid2', 'link']
print(df_test['link'])

df_test = df_test[(df_test['nodeid1'] < 10000) & (df_test['nodeid2'] < 10000)]
print(df_test['link'])
#print(df_test)
df_test_1 = df_test[['nodeid1', 'nodeid2']]
#print(df_test_1)
list_1_emb = []
list_2_emb = []
list_no_emb = []
#print(dict1.keys())
for index, row in df_test_1.iterrows():

  if (row['nodeid1'] in dict1.keys()):
    list_1_emb.append((E1[dict1[row['nodeid1']], :]))
  elif (row['nodeid1'] in dict2.keys()):
    list_1_emb.append((E2[dict2[row['nodeid1']], :])) 
  elif (row['nodeid1'] in dict3.keys()):
    list_1_emb.append((E3[dict3[row['nodeid1']], :]))
  elif (row['nodeid1'] in dict4.keys()):
    list_1_emb.append((E4[dict4[row['nodeid1']], :]))     
  else:
    
  
    
    list_no_emb.append(index)  
  if (row['nodeid2'] in dict1.keys()):
    list_2_emb.append((E1[dict1[row['nodeid2']], :])) 
 
  elif (row['nodeid2'] in dict2.keys()):
    list_2_emb.append((E2[dict2[row['nodeid2']], :]))
  
  elif (row['nodeid2'] in dict3.keys()):
    list_2_emb.append((E3[dict3[row['nodeid2']], :]))
  
  elif (row['nodeid2'] in dict4.keys()):
    list_2_emb.append((E4[dict4[row['nodeid2']], :])) 
  else:
    
    
    
    list_no_emb.append(index) 

print(len(list_1_emb))
print(len(list_2_emb))
print((list_no_emb))
#LIST_INT_EMB1_ind = list(set(list_1_emb) & set(list(df_test['nodeid1'])))
#LIST_INT_EMB2_ind = list(set(list_2_emb) & set(list(df_test['nodeid2'])))
#List_ind_common = list(set(LIST_INT_EMB1_ind) & set(LIST_INT_EMB2_ind))
#LIST_INT_EMB1_ind = set(list(list_1_emb)).intersection(list(df_test['nodeid1'])).index.values
#print(List_ind_common)
print(df_test_1.shape)
print('ddnssnd')
df_test_1_new = df_test_1.drop(list_no_emb)
print(df_test_1_new.shape)
print(df_test_1_new)
list_1_emb_ = []
list_2_emb_ = []
IND_TEST = list(df_test_1_new.index.values)
for index, row in df_test_1_new.iterrows():

  if (row['nodeid1'] in dict1.keys()):
    list_1_emb_.append((E1[dict1[row['nodeid1']], :]))
  
  elif (row['nodeid1'] in dict2.keys()):
    list_1_emb_.append((E2[dict2[row['nodeid1']], :])) 
   
  elif (row['nodeid1'] in dict3.keys()):
    list_1_emb_.append((E3[dict3[row['nodeid1']], :]))
   
  elif (row['nodeid1'] in dict4.keys()):
    list_1_emb_.append((E4[dict4[row['nodeid1']], :]))   
   
  else:
    
  
    
    list_no_emb_.append(index)  
  if (row['nodeid2'] in dict1.keys()):
    list_2_emb_.append((E1[dict1[row['nodeid2']], :])) 
    
  elif (row['nodeid2'] in dict2.keys()):
    list_2_emb_.append((E2[dict2[row['nodeid2']], :]))
    
  elif (row['nodeid2'] in dict3.keys()):
    list_2_emb_.append((E3[dict3[row['nodeid2']], :]))
   
  elif (row['nodeid2'] in dict4.keys()):
    list_2_emb_.append((E4[dict4[row['nodeid2']], :])) 
   
  else:
    list_no_emb_.append(index) 

    
    
    
    list_no_emb_.append(index)   
  
print(len(list_1_emb_))
print(len(list_2_emb_))
print(len(IND_TEST))


print('3879' in dict4.keys())

0        1
1        1
2        1
3        1
4        1
        ..
17045    0
17046    0
17047    0
17048    0
17049    0
Name: link, Length: 17050, dtype: int64
2       1
4       1
5       1
6       1
12      1
       ..
9767    0
9771    0
9773    0
9783    0
9785    0
Name: link, Length: 363, dtype: int64
318
245
[12, 28, 76, 105, 108, 161, 195, 206, 263, 324, 332, 366, 375, 380, 398, 425, 505, 553, 559, 568, 568, 569, 592, 658, 674, 723, 726, 788, 788, 790, 796, 798, 836, 885, 906, 967, 1002, 1012, 1107, 1138, 1141, 1185, 1199, 1204, 1223, 1241, 8530, 8533, 8537, 8540, 8540, 8543, 8548, 8550, 8552, 8559, 8575, 8581, 8590, 8594, 8601, 8601, 8615, 8618, 8624, 8638, 8643, 8675, 8693, 8707, 8733, 8737, 8748, 8756, 8760, 8768, 8772, 8775, 8798, 8798, 8801, 8811, 8816, 8825, 8828, 8833, 8864, 8867, 8879, 8917, 8925, 8929, 8958, 8973, 8975, 8978, 8992, 8995, 9007, 9009, 9012, 9012, 9018, 9033, 9042, 9050, 9050, 9064, 9090, 9162, 9184, 9186, 9187, 9215, 9245, 9245, 9256, 9259, 9272, 9277, 9

CREATING A DATAFRAME FROM THE LIST OF EMBEDDING VECTORS AND FITTING THE MODEL
SAMPLING A NUMBER OF ROWS FROM THE TRAINING SET RANDOMLY SUCH THAT THE RATIO OF TRAIN/TEST = 4/1


In [ ]:
LIST_EMB_TEST = pd.DataFrame({'embed_nodeid1' : list_1_emb_ , 'embed_nodeid2' : list_2_emb_ })
LIST_EMB_TEST['Had_prod'] = LIST_EMB_TEST['embed_nodeid1'] * LIST_EMB_TEST['embed_nodeid2']
#print(LIST_EMB_TEST['Had_prod'])
LIST_EMB_TEST_ = LIST_EMB_TEST.Had_prod.apply(pd.Series)
#print(LIST_EMB_TEST_)
LIST_EMB_TEST_ = LIST_EMB_TEST_.to_numpy()
#list_1_emb = (set(list(list_1_emb)) - set(list(list_1_emb_)))
#list_2_emb = (set(list_2_emb) - set(list_2_emb_))
for i in list_1_emb_: 
    try: 
        list1_embold.remove(i) 
    except ValueError: 
        pass
for i in list_2_emb_: 
    try: 
        list2_embold.remove(i) 
    except ValueError: 
        pass        
print(len(list1_embold))
print(len(list2_embold))
EMB_DATA_new = pd.DataFrame({'embedid1': list1_embold, 'embedid2': list2_embold})
print(EMB_DATA_new. shape)

EMB_DATA_new['Had_prod'] = EMB_DATA_new['embedid1']*EMB_DATA_new['embedid2']
EMB_DATA1 = pd.DataFrame({'Had_EMB_LIST' : list(EMB_DATA_new['Had_prod']), 'Link' : list(DATA['link'])})


print(EMB_DATA1.columns)
LISTEMB_ = []







14360
14360
(14360, 2)
Index(['Had_EMB_LIST', 'Link'], dtype='object')


AVERAGING OVER ALL THE PREDICTIONS

In [ ]:
X_ = LIST_EMB_TEST_

pred1 = clf1.predict(X_)
pred2 = clf2.predict(X_)
pred3 = clf3.predict(X_)
pred4 = clf4.predict(X_)
pred5 = clf5.predict(X_)
Y_ = list(df_test.loc[IND_TEST, 'link'])
print(Y_)

SCORE1 = ((pred1 == Y_).sum()/len(Y_))
SCORE2 = ((pred2 == Y_).sum()/len(Y_))
SCORE3 = ((pred3 == Y_).sum()/len(Y_))
SCORE4 = ((pred4 == Y_).sum()/len(Y_))
SCORE5 = ((pred5 == Y_).sum()/len(Y_))
print(SCORE3)




#print(Y)
#pred = clf.predict(X_)



#print((pred))
#print(pred == Y_)
#print(len(set(pred).intersection(Y_)))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.5943396226415094


In [ ]:
SCORE1

0.5943396226415094

NODE CLASSIFICATION

In [ ]:


DF_NODE = pd.read_csv('/content/label.dat',sep = '\t', header = None)
print(DF_NODE.shape)
DF_NODE.columns = ['nodeid', 'nodename', 'nodetype', 'nodelabel']
DF_NODE = DF_NODE.drop(['nodename', 'nodetype'], axis = 1)
print(DF_NODE.columns)
DF_NODE = DF_NODE[DF_NODE['nodeid'] < 10000]
print(set(node_list).intersection(list(DF_NODE['nodeid'])))
#print(DF_NODE)
DF_NODE_TEST = pd.read_csv('/content/label.dat.test', sep = '\t', header = None)
DF_NODE_TEST.columns = ['nodeid', 'nodename', 'nodetype', 'nodelabel']
print(DF_NODE_TEST[DF_NODE_TEST['nodeid'] < 10000].shape)
DF_NODE_TEST = DF_NODE_TEST[DF_NODE_TEST['nodeid'] < 10000]

(368, 4)
Index(['nodeid', 'nodelabel'], dtype='object')
{8832, 5385, 5263, 1168, 9489, 6162, 6034, 8595, 2464, 5923, 5412, 9897, 9770, 5811, 1470, 9158, 719, 6484, 5213, 3297, 4580, 9971, 7035, 5374}


FileNotFoundError: ignored

In [ ]:
DF_NODE.shape

In [ ]:


node_emb_list = []
node_emb_absent_list = []
NODE_IDX = []
for index, row in DF_NODE.iterrows():
  if (row['nodeid'] in dict1.keys()):

   
    node_emb_list.append((E1[dict1[row['nodeid']], :]))
    NODE_IDX.append(index)
  
  elif (row['nodeid'] in dict2.keys()):
    node_emb_list.append((E2[dict2[row['nodeid']], :])) 
    NODE_IDX.append(index)
  
   
  elif (row['nodeid'] in dict3.keys()):
    node_emb_list.append((E3[dict3[row['nodeid']], :]))
    NODE_IDX.append(index)
  
   
  elif (row['nodeid'] in dict4.keys()):
    node_emb_list.append((E4[dict4[row['nodeid']], :])) 
    NODE_IDX.append(index)
    
   
  else:
    
  
    
    node_emb_absent_list.append(index)

print(len(node_emb_list)) 
NODE_CLASSIFICATION_DF = pd.DataFrame({'node_embed': node_emb_list})
NODE_CLASSIFICATION_DF = NODE_CLASSIFICATION_DF.node_embed.apply(pd.Series)
#print(NODE_CLASSIFICATION_DF)

#print(node_emb_absent_list)
LABEL_NODE = DF_NODE.loc[NODE_IDX, 'nodelabel']
NODE_X = NODE_CLASSIFICATION_DF.to_numpy()
NODE_Y = list(LABEL_NODE)

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC


clf = OneVsRestClassifier(SVC(kernel='linear', probability=True))
clf.fit(NODE_X, NODE_Y)
print(DF_NODE_TEST.shape)
node_test_embed_cl = []
node_emb_absent_list_test = []
FINAL_INDEX = []
for index, row in DF_NODE_TEST.iterrows():
  if (row['nodeid'] in dict1.keys()):

   
    node_test_embed_cl.append((E1[dict1[row['nodeid']], :]))
    FINAL_INDEX.append(index)
    
  
  elif (row['nodeid'] in dict2.keys()):
    node_test_embed_cl.append((E2[dict2[row['nodeid']], :])) 
    FINAL_INDEX.append(index)
    
  
   
  elif (row['nodeid'] in dict3.keys()):
    node_test_embed_cl.append((E3[dict3[row['nodeid']], :]))
    FINAL_INDEX.append(index)
    
  
   
  elif (row['nodeid'] in dict4.keys()):
    node_test_embed_cl.append((E4[dict4[row['nodeid']], :])) 
    FINAL_INDEX.append(index)
    
    
   
  else:
    
  
    
    node_emb_absent_list_test.append(index)



print(len(node_test_embed_cl))
FINAL_NODE_TEST = pd.DataFrame({'had_prod' : node_test_embed_cl})
FINAL_NODE_TEST1 = FINAL_NODE_TEST.had_prod.apply(pd.Series)
X_FINAL = FINAL_NODE_TEST1.to_numpy()
Y_FINAL= list(DF_NODE_TEST.loc[FINAL_INDEX, 'nodelabel'])
print(Y_FINAL)

clf.predict(X_FINAL)


